In [20]:
import bz2
import codecs
import itertools
import json
import pandas as pd
import pickle

with open('2016-500k.pickle', 'rb') as f:
    subset = pickle.load(f)
comments = subset['body'].values
print(comments[-3:])

['-- John Madden' "I don't know man, Raw Data is sick :)"
 "Don't freak out; your post has **not** been removed. We just wanted to remind you of a few important rules:\n\n* You must state where and from whom you obtained your Pokémon, to the best of your knowledge, as well as any other important information regarding their origin or trade history.\n* The OT and ID No. must be listed for any events or shinies without a set OT/ID No. For more details, please refer to [Rule 4](/r/pokemontrades/wiki/rules).\n* Valuable Pokémon obtained from the GTS, Wonder Trade, or Passerby trades are not allowed to be traded.\n* Events transferred from previous generations can only be traded in Event3-5 tagged posts with full details listed. Please read [Rule 9](/r/pokemontrades/wiki/rules) for more information.\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/pokemontrades) if you have any questions or concerns.*"]


In [32]:
comments_clean = [s.translate(str.maketrans('', '', string.punctuation)).replace('\n', ' ') for s in comments ]
comments_clean[-3:]

[' John Madden',
 'I dont know man Raw Data is sick ',
 'Dont freak out your post has not been removed We just wanted to remind you of a few important rules   You must state where and from whom you obtained your Pokémon to the best of your knowledge as well as any other important information regarding their origin or trade history  The OT and ID No must be listed for any events or shinies without a set OTID No For more details please refer to Rule 4rpokemontradeswikirules  Valuable Pokémon obtained from the GTS Wonder Trade or Passerby trades are not allowed to be traded  Events transferred from previous generations can only be traded in Event35 tagged posts with full details listed Please read Rule 9rpokemontradeswikirules for more information  I am a bot and this action was performed automatically Please contact the moderators of this subredditmessagecomposetorpokemontrades if you have any questions or concerns']

In [36]:
# Simple stop words
stop_words = ["a", "able", "about", "above", "according", "accordingly", "across", "actually", "after", "afterwards", "again", "against", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway", "anyways", "anywhere", "apart", "appear", "appreciate", "appropriate", "are", "around", "as", "aside", "ask", "asking", "associated", "at", "available", "away", "awfully", "b", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "both", "brief", "but", "by", "c", "came", "can", "cannot", "cant", "cause", "causes", "certain", "certainly", "changes", "clearly", "co", "com", "come", "comes", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "course", "currently", "d", "definitely", "described", "despite", "did", "different", "do", "does", "doing", "done", "down", "downwards", "during", "e", "each", "edu", "eg", "eight", "either", "else", "elsewhere", "enough", "entirely", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "f", "far", "few", "fifth", "first", "five", "followed", "following", "follows", "for", "former", "formerly", "forth", "four", "from", "further", "furthermore", "g", "get", "gets", "getting", "given", "gives", "go", "goes", "going", "gone", "got", "gotten", "greetings", "h", "had", "happens", "hardly", "has", "have", "having", "he", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "hi", "him", "himself", "his", "hither", "hopefully", "how", "howbeit", "however", "i", "ie", "if", "ignored", "immediate", "in", "inasmuch", "inc", "indeed", "indicate", "indicated", "indicates", "inner", "insofar", "instead", "into", "inward", "is", "it", "its", "itself", "j", "just", "k", "keep", "keeps", "kept", "know", "knows", "known", "l", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "like", "liked", "likely", "little", "look", "looking", "looks", "ltd", "m", "mainly", "many", "may", "maybe", "me", "mean", "meanwhile", "merely", "might", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "n", "name", "namely", "nd", "near", "nearly", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "no", "nobody", "non", "none", "noone", "nor", "normally", "not", "nothing", "novel", "now", "nowhere", "o", "obviously", "of", "off", "often", "oh", "ok", "okay", "old", "on", "once", "one", "ones", "only", "onto", "or", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "own", "p", "particular", "particularly", "per", "perhaps", "placed", "please", "plus", "possible", "presumably", "probably", "provides", "q", "que", "quite", "qv", "r", "rather", "rd", "re", "really", "reasonably", "regarding", "regardless", "regards", "relatively", "respectively", "right", "s", "said", "same", "saw", "say", "saying", "says", "second", "secondly", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "shall", "she", "should", "since", "six", "so", "some", "somebody", "somehow", "someone", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specified", "specify", "specifying", "still", "sub", "such", "sup", "sure", "t", "take", "taken", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "theres", "thereupon", "these", "they", "think", "third", "this", "thorough", "thoroughly", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "twice", "two", "u", "un", "under", "unfortunately", "unless", "unlikely", "until", "unto", "up", "upon", "us", "use", "used", "useful", "uses", "using", "usually", "uucp", "v", "value", "various", "very", "via", "viz", "vs", "w", "want", "wants", "was", "way", "we", "welcome", "well", "went", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "willing", "wish", "with", "within", "without", "wonder", "would", "would", "x", "y", "yes", "yet", "you", "your", "yours", "yourself", "yourselves", "z", "zero"]

# Parse text into words, make lowercase and remove stop words
txts = [[word for word in sentence.lower().split() if word not in stop_words]
        for sentence in comments_clean]

print("counting words...")
from collections import Counter
frequency = Counter([word for txt in txts for word in txt])

# Now grab tokens that appear more than once
tokens = [[token for token in txt if frequency[token] > 2]
          for txt in txts]

counting words...


In [37]:
frequency.most_common(100)

[('dont', 50392),
 ('im', 49353),
 ('people', 39460),
 ('time', 30520),
 ('good', 29725),
 ('make', 23449),
 ('deleted', 22531),
 ('youre', 22056),
 ('ive', 18278),
 ('game', 17507),
 ('back', 17069),
 ('god', 16449),
 ('doesnt', 15374),
 ('didnt', 15327),
 ('work', 15171),
 ('pretty', 14942),
 ('thing', 14472),
 ('gt', 14297),
 ('lot', 14208),
 ('removed', 13664),
 ('things', 13469),
 ('2', 12479),
 ('great', 12187),
 ('feel', 12072),
 ('post', 11831),
 ('love', 11595),
 ('day', 11587),
 ('isnt', 11581),
 ('id', 11542),
 ('hes', 11089),
 ('shit', 11086),
 ('years', 11038),
 ('yeah', 10954),
 ('made', 10583),
 ('point', 10496),
 ('find', 10487),
 ('bad', 10388),
 ('1', 9887),
 ('year', 9792),
 ('play', 9764),
 ('ill', 9742),
 ('3', 9451),
 ('long', 9373),
 ('give', 9082),
 ('theyre', 8957),
 ('man', 8706),
 ('life', 8358),
 ('thought', 8324),
 ('fuck', 8167),
 ('put', 8103),
 ('makes', 8091),
 ('part', 7952),
 ('guy', 7920),
 ('money', 7917),
 ('lol', 7785),
 ('hard', 7645),
 ('big', 7